In [ ]:
# default_exp core

# Core utilities module

> This module implements core modules for the exercies.

In [ ]:
#hide
from nbdev.showdoc import *
import pandas as pd
import os
import sys
import nltk
#from sklearn.model_selection import train_test_split
import string
import re
import nltk
from multiprocessing import  Pool
import numpy as np

In [ ]:
#export
def read_sentiment_dataset(path):
    '''
    Reads the sentiment140 dataset csv file. The dataset can be downloaded from Kaggle
    '''
    return pd.read_csv(path,engine='python', header=None ,names=['target', 'ids', 'date','flag','user','text'])

In [ ]:
#export
def subsample_dataset(df,no_lines=5000):
    
    "Subsamples the dataset for faster processing"
    df_neg = df[df.target == 0]
    df_pos = df[df.target == 4]

    df_pos = df_pos[:no_lines]
    df_neg = df_neg[:no_lines]

    return pd.concat([df_pos,df_neg])

In [ ]:
#export
def preprocess_text(text):
    '''
    Preprocess tweets to remove https links and mentions
    Remove punctioation
    Stemmize the words
    '''
    
    # Make lowercase
    text = text.lower()
    #print(text)

    # Remove mentions and http links
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)

    # Remove punctionation
    text = ''.join(char for char in text if char not in string.punctuation)

    # Seperate text into words
    words = nltk.word_tokenize(text)

    # Stemmize
    porter = nltk.stem.porter.PorterStemmer()
    stemmed = [porter.stem(word) for word in words]
    
    
    #print(words)

    #print(stemmed)

    return stemmed
    

In [ ]:
#export
def parallelize(data, func, num_of_processes=8):
    '''
    Parallelize the dataframe processing
    '''
    
    data_split = np.array_split(data, num_of_processes)
    pool = Pool(num_of_processes)
    data = pool.map(func, data_split)
    pool.close()
    pool.join()
    return data

In [ ]:
#export
def preprocess_df_text(df):
    '''
    Preprocesses the text field of dataframe
    '''
    data = []
    for index,item in df.iterrows():
        data.extend(preprocess_text(item.text))
    return data

In [ ]:
#export
def calculate_word_attributes(df):
    '''
    Calculates the word attributes in a dataframe such as 
    total words, unique words and word frequency dictionary
    '''

    word_lists=parallelize(df,preprocess_df)
    all_words = []
    for item in word_lists:
        all_words.extend(item)
    
    unique_all_words = list(set(all_words))
    
    word_freq_dict = {}
    for item in unique_all_words:
        word_freq_dict[item] = all_words.count(item)
    
    return all_words,unique_all_words,word_freq_dict